# 🖥️ Mimicking an Argo float

This tutorial shows how simple it is to construct a Kernel in Parcels that mimics the [vertical movement of Argo floats](https://www.aoml.noaa.gov/phod/argo/images/argo_float_mission.jpg).

We first define the kernels for each phase of the Argo cycle.

In [ ]:
import numpy as np

# Define the new Kernel that mimics Argo vertical movement
driftdepth = 1000  # maximum depth in m
maxdepth = 2000  # maximum depth in m
vertical_speed = 0.10  # sink and rise speed in m/s
cycletime = 10 * 86400  # total time of cycle in seconds
drifttime = 9 * 86400  # time of deep drift in seconds


def ArgoVerticalMovement(particles, fieldset):
    # Split particles based on their current cycle_phase
    ptcls0 = particles[particles.cycle_phase == 0]
    ptcls1 = particles[particles.cycle_phase == 1]
    ptcls2 = particles[particles.cycle_phase == 2]
    ptcls3 = particles[particles.cycle_phase == 3]
    ptcls4 = particles[particles.cycle_phase == 4]

    # Phase 0: Sinking with vertical_speed until depth is driftdepth
    ptcls0.dz += vertical_speed * ptcls0.dt
    ptcls0.cycle_phase = np.where(
        ptcls0.z + ptcls0.dz >= driftdepth, 1, ptcls0.cycle_phase
    )
    ptcls0.dz = np.where(
        ptcls0.z + ptcls0.dz >= driftdepth, driftdepth - ptcls0.z, ptcls0.dz
    )

    # Phase 1: Drifting at depth for drifttime seconds
    ptcls1.drift_age += ptcls1.dt
    ptcls1.cycle_phase = np.where(ptcls1.drift_age >= drifttime, 2, ptcls1.cycle_phase)
    ptcls1.drift_age = np.where(ptcls1.drift_age >= drifttime, 0, ptcls1.drift_age)

    # Phase 2: Sinking further to maxdepth
    ptcls2.dz += vertical_speed * ptcls2.dt
    ptcls2.cycle_phase = np.where(
        ptcls2.z + ptcls2.dz >= maxdepth, 3, ptcls2.cycle_phase
    )
    ptcls2.dz = np.where(
        ptcls2.z + ptcls2.dz >= maxdepth, maxdepth - ptcls2.z, ptcls2.dz
    )

    # Phase 3: Rising with vertical_speed until at surface
    ptcls3.dz -= vertical_speed * ptcls3.dt
    ptcls3.temp = fieldset.thetao[ptcls3.time, ptcls3.z, ptcls3.lat, ptcls3.lon]
    ptcls3.cycle_phase = np.where(
        ptcls3.z + ptcls3.dz <= fieldset.mindepth, 4, ptcls3.cycle_phase
    )
    ptcls3.dz = np.where(
        ptcls3.z + ptcls3.dz <= fieldset.mindepth,
        fieldset.mindepth - ptcls3.z,
        ptcls3.dz,
    )

    # Phase 4: Transmitting at surface until cycletime is reached
    ptcls4.cycle_phase = np.where(ptcls4.cycle_age >= cycletime, 0, ptcls4.cycle_phase)
    ptcls4.cycle_age = np.where(ptcls4.cycle_age >= cycletime, 0, ptcls4.cycle_age)
    ptcls4.temp = np.nan  # no temperature measurement when at surface

    particles.cycle_age += particles.dt  # update cycle_age

And then we can run Parcels with this 'custom kernel'.

Below we use the horizontal velocity fields of CopernicusMarine, which are provided as example_data with Parcels.


In [ ]:
from datetime import timedelta

import xarray as xr

import parcels

# Load the CopernicusMarine data in the Agulhas region from the example_datasets
example_dataset_folder = parcels.download_example_dataset(
    "CopernicusMarine_data_for_Argo_tutorial"
)

ds_fields = xr.open_mfdataset(f"{example_dataset_folder}/*.nc", combine="by_coords")

# TODO check how we can get good performance without loading full dataset in memory
ds_fields.load()  # load the dataset into memory

# Select fields
fields = {
    "U": ds_fields["uo"],
    "V": ds_fields["vo"],
    "thetao": ds_fields["thetao"],
}

# Convert to SGRID-compliant dataset and create FieldSet
ds_fset = parcels.convert.copernicusmarine_to_sgrid(fields=fields)
fieldset = parcels.FieldSet.from_sgrid_conventions(ds_fset)
fieldset.add_constant("mindepth", 1.0)

# Define a new Particle type including extra Variables
ArgoParticle = parcels.Particle.add_variable(
    [
        parcels.Variable("cycle_phase", dtype=np.int32, initial=0.0),
        parcels.Variable("cycle_age", dtype=np.float32, initial=0.0),
        parcels.Variable("drift_age", dtype=np.float32, initial=0.0),
        parcels.Variable("temp", dtype=np.float32, initial=np.nan),
    ]
)

# Initiate one Argo float in the Agulhas Current
pset = parcels.ParticleSet(
    fieldset=fieldset,
    pclass=ArgoParticle,
    lon=[32],
    lat=[-31],
    z=[fieldset.mindepth],
)

# combine Argo vertical movement kernel with built-in Advection kernel
kernels = [
    ArgoVerticalMovement,
    parcels.kernels.AdvectionRK4,
]

# Create a ParticleFile object to store the output
output_file = parcels.ParticleFile(
    store="argo_float.zarr",
    outputdt=timedelta(minutes=15),
    chunks=(1, 500),  # setting to write in chunks of 500 observations
)

# Now execute the kernels for 30 days, saving data every 30 minutes
pset.execute(
    kernels,
    runtime=timedelta(days=30),
    dt=timedelta(minutes=15),
    output_file=output_file,
)

Now we can plot the trajectory of the Argo float with some simple calls to netCDF4 and matplotlib.

First plot the depth as a function of time, with the temperature as color (only on the upcast).

In [ ]:
ds_particles = xr.open_zarr(output_file.store)
x = ds_particles["lon"][:].squeeze()
y = ds_particles["lat"][:].squeeze()
z = ds_particles["z"][:].squeeze()
time = ds_particles["time"][:].squeeze()
temp = ds_particles["temp"][:].squeeze()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(13, 6))
ax = plt.axes()
ax.plot(time, z, color="gray")
cb = ax.scatter(time, z, c=temp, s=20, marker="o", zorder=2)
ax.set_xlabel("Time [days]")
ax.set_ylabel("Depth (m)")
ax.invert_yaxis()
fig.colorbar(cb, label="Temperature (°C)")
plt.show()

We can also make a 3D plot of the trajectory colored by temperature.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(13, 8))
ax = plt.axes(projection="3d")
ax.view_init(azim=-145)
ax.plot3D(x, y, z, color="gray")
cb = ax.scatter(x, y, z, c=temp, s=20, marker="o", zorder=2)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_zlabel("Depth (m)")
ax.set_zlim(np.max(z), 0)
fig.colorbar(cb, label="Temperature (°C)")
plt.show()